In [1]:
# from newsrec.config import model_name
import pandas as pd
import swifter
import json
import math
from tqdm import tqdm
from os import path
from pathlib import Path
import random
from nltk.tokenize import word_tokenize
import numpy as np
import csv
import importlib
from transformers import RobertaTokenizer, RobertaModel
import torch

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
train_dir = '../../data/train'
source = f'../../data/glove/glove.840B.{300}d.txt'
target = path.join(train_dir, 'pretrained_word_embedding.npy')
word2int_path = path.join(train_dir, 'word2int.tsv')

In [7]:
word2int = pd.read_table(word2int_path, na_filter=False, index_col='word')
source_embedding = pd.read_table(source,
                                 index_col=0,
                                 sep=' ',
                                 header=None,
                                 quoting=csv.QUOTE_NONE,
                                 names=range(300))

In [8]:
word2int.head()

,int
word,
the,1
brands,2
queen,3
elizabeth,4
",",5


In [18]:
word2int.tail()

,int
word,
chattahoochee-oconee,101216
ripely,101217
simmonds,101218
koeller,101219
"1,221",101220


In [10]:
source_embedding.index.rename('word', inplace=True)

In [11]:
source_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
word,,,,,,,,,,,,,,,,,,,,,
",",-0.082752,0.672040,-0.14987,-0.064983,0.056491,0.402280,0.002775,-0.331100,-0.306910,2.0817,...,-0.14331,0.018267,-0.18643,0.207090,-0.355980,0.053380,-0.050821,-0.191800,-0.378460,-0.06589
.,0.012001,0.207510,-0.12578,-0.593250,0.125250,0.159750,0.137480,-0.331570,-0.136940,1.7893,...,0.16165,-0.066737,-0.29556,0.022612,-0.281350,0.063500,0.140190,0.138710,-0.360490,-0.03500
the,0.272040,-0.062030,-0.18840,0.023225,-0.018158,0.006719,-0.138770,0.177080,0.177090,2.5882,...,-0.42810,0.168990,0.22511,-0.285570,-0.102800,-0.018168,0.114070,0.130150,-0.183170,0.13230
and,-0.185670,0.066008,-0.25209,-0.117250,0.265130,0.064908,0.122910,-0.093979,0.024321,2.4926,...,-0.59396,-0.097729,0.20072,0.170550,-0.004736,-0.039709,0.324980,-0.023452,0.123020,0.33120
to,0.319240,0.063160,-0.27858,0.261200,0.079248,-0.214620,-0.104950,0.154950,-0.033530,2.4834,...,-0.12977,0.371300,0.18888,-0.004274,-0.106450,-0.258100,-0.044629,0.082745,0.097801,0.25045


In [17]:
source_embedding.tail()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
word,,,,,,,,,,,,,,,,,,,,,
xtremecaffeine,0.073032,-1.02940,-0.015436,0.72615,-0.99246,0.29766,0.31656,0.20248,0.35499,-1.5233,...,0.14481,-0.475240,-0.013947,0.561070,0.47501,0.12738,0.12957,0.422640,-0.11333,0.16125
yildirim,0.222760,-0.29639,0.694120,0.19362,-0.31276,0.24824,0.43449,-1.58940,0.24365,-1.6554,...,1.10830,-0.009433,-0.185360,0.258170,0.86129,-0.41398,0.24018,0.093864,-0.16545,-0.34593
z/28,0.734400,-0.33641,0.269180,0.41843,-0.18900,0.28020,0.23875,-0.12879,-0.15797,-1.2841,...,-0.31101,-0.989550,0.276270,0.252650,0.47931,-0.48432,-1.00550,0.637180,-0.13914,-0.16472
zipout,0.212150,-0.99456,1.178200,2.07210,-0.44271,0.58730,0.26128,-0.42554,0.03091,-1.6743,...,-0.19946,-0.198480,1.091500,-0.351070,-1.04720,-0.41382,-0.21139,0.934270,-0.93286,-0.51479
zulchzulu,-0.079690,-0.22905,0.803660,-0.78865,-0.40567,-0.15716,-0.42302,0.64081,-0.13215,-1.4109,...,0.64657,-0.415910,0.142400,-0.051749,0.38925,-0.20522,0.26878,-0.083561,0.48532,-0.73130


In [12]:
merged = word2int.merge(source_embedding,
                        how='inner',
                        left_index=True,
                        right_index=True)
merged.set_index('int', inplace=True)

In [13]:
merged.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
int,,,,,,,,,,,,,,,,,,,,,
854,-0.26554,0.33531,0.218600,-0.301000,-0.05547,-0.242360,0.172360,-0.163340,-0.109000,1.26710,...,-0.38645,-0.150560,-0.032827,-0.10529,0.28397,-0.255000,0.15195,-0.178590,-0.062878,0.16232
4779,-0.44467,0.69536,0.427480,0.219060,0.11757,-0.213110,0.580860,0.073145,-0.085245,0.36655,...,0.28116,0.357600,-0.056253,0.21949,0.35125,-0.413770,-0.27868,-0.079391,-0.527170,-0.12413
517,-0.60712,0.42544,0.510400,-0.287500,0.51475,0.082824,-0.415860,-0.343850,0.489610,0.70518,...,-0.30249,-0.025315,0.471890,-0.22638,-0.97497,0.061226,-0.38895,-0.185880,0.025965,-0.48231
3117,-1.25750,0.71648,-0.065293,0.006738,-0.27967,-0.068012,-0.093761,0.316350,0.159900,0.77747,...,0.15569,0.349450,0.160910,-0.29625,-0.32933,-0.447430,-0.86472,-0.375350,0.111350,-0.33226
945,-0.32971,0.33465,-0.067601,-0.032601,0.75784,-0.173780,0.321860,-0.079652,0.182100,0.66829,...,-0.71663,-0.158360,0.018979,0.27348,0.23848,0.249300,0.40164,-0.264940,0.166040,0.22047


In [15]:
missed_index = np.setdiff1d(np.arange(len(word2int) + 1),
                            merged.index.values)
missed_embedding = pd.DataFrame(data=np.random.normal(
    size=(len(missed_index), 300)))
missed_embedding['int'] = missed_index
missed_embedding.set_index('int', inplace=True)

In [16]:
final_embedding = pd.concat([merged, missed_embedding]).sort_index()
np.save(target, final_embedding.values)

print(
    f'Rate of word missed in pretrained embedding: {(len(missed_index)-1)/len(word2int):.4f}'
)

Rate of word missed in pretrained embedding: 0.2958
